# TP2 : Extraction d'informations

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- de se familiariser avec les techniques d’enrichissement (extraction de mots-clés, reconnaissance d’entités nommées, analyse de sentiment...)
- d’appliquer celles-ci au corpus de bulletins de la ville de Bruxelles préalablement convertis au format .txt

Pour ce faire, vous utiliserez différentes librairies Python vues au cours.

## 1. Keywords

### Librairies utilisées dans le code

#### Choix d'une année à analyser

In [1]:
import os
from collections import defaultdict, Counter
import textract
import nltk
from nltk.corpus import stopwords
import yake
from wordcloud import WordCloud
from IPython.display import Image

### Sélection des fichiers à analyser

#### Choix d'une année à analyser

In [2]:
selected_year = 1930

#### Stocker la liste des fichiers contenus dans le dossier 'txt'

In [3]:
txt_path = "../data/txt/"
files = sorted(os.listdir(txt_path))
len(files)

2827

#### Lister les fichiers pour Bruxelles et pour Laeken

In [4]:
bxl_selected_year = [f for f in files if f.startswith('Bxl_') and str(selected_year) in f]
print(len(bxl_selected_year))

lkn_selected_year = [f for f in files if f.startswith('Lkn_') and str(selected_year) in f]
print(len(lkn_selected_year))

31
0


#### Vérification du nombre de fichiers

In [5]:
count_years = defaultdict(int)

for f in files: 
    if "_" in f and f.endswith("txt"): 
        elems = f.split("_") 
        year = elems[1] 
        count_years[year] += 1 
        
n_selected_year = count_years[year]
print(n_selected_year)

35


### Récupérer le texte des fichiers

#### Créer le dossier adéquat

In [ ]:
path_1930 = '../data/1930'
if not os.path.exists(path_1930):
    os.mkdir(path_1930)

#### Copier les fichiers désirés dans le dossier

In [ ]:
for txt in bxl_1930:
    text = textract.process(os.path.join(txt_path, txt))
    with open(os.path.join(path_1930, txt), 'wb') as f: 
        f.write(text) 

#### Concaténer le texte des fichiers

In [ ]:
!cat ../data/{year}/*.txt > ../data/{year}.txt

#### Compter le nombre de mots repris dans le fichier

In [ ]:
!wc -m ../data/{year}.txt

In [ ]:
texts_1930 = open(os.path.join(f"../data/", "1930.txt"), 'r').read()
texts_1930[:500]

### Extraction des mots clés

#### Instancier l'extracteur de mots clés

In [ ]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

#### Extraire les mots clés pour l'année entière

In [ ]:
keywords = kw_extractor.extract_keywords(texts_1930.lower()) 
keywords

#### Ajout de mots à ignorer

In [ ]:
ignored = ["conseil communal", "conseil général", "bruxelles"]
ignored

#### Ne garder que les bigrammes

In [ ]:
bigram = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2 and kw.lower() not in ignored:
        sw1.append(kw)

print(sorted(bigram))

#### Ajouter les mongrammes à la liste des stopwords

#### Analyse séquentielle en fonction des tomes

In [ ]:
for f in sorted(bxl_1930):
    text = open(os.path.join(path_1930, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower()) # rajouter le lower 
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...\n")

### Nettoyer le fichier

#### Tokenisation

In [ ]:
words = nltk.wordpunct_tokenize(texts_1930)
print(f"{len(words)} words found")

print(words[:200])

#### Création de la 'stopwrods' de base enrichie

In [ ]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", 
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu"]

#### Calculer la taille du vocabulaire

In [ ]:
words_cleaned = []

for w in words:
    if len(w) > 2 and w.isalpha() and w.lower() not in sw:
        words_cleaned.append(w.lower())
        
voc = set(words_cleaned)

print(f"{len(words_cleaned)} words kept ({len(voc)} different word forms)")
print(words_cleaned[:200])

OU

In [ ]:
words_cleaned = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(words_cleaned)
print(f"{len(words_cleaned)} words kept ({len(voc)} different word forms)")

#### Récupérer les mots les plus fréquents

In [ ]:
fdist = nltk.FreqDist(words_cleaned)
fdist.most_common(10)

#### Ajout d'une liste de mots à ignorer

In [ ]:
sw += ["francs", "ville", "bruxelles", "conseil", "service", "conseil"] 

print(sorted(sw))

In [ ]:
sample_1 = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(sample_1)
print(f"{len(sample_1)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(sample_1)
fdist.most_common(10)

In [ ]:
sw += ["collège", "communal", "bourgmestre"] 

print(sorted(sw))

In [ ]:
sample_2 = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(sample_2)
print(f"{len(sample_2)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(sample_2)
fdist.most_common(20)

In [ ]:
sw += ["total", "echevin", "membres", "divers", "année", "article", "question", "diverses", "messieurs", "art"] 

print(sorted(sw))

### Enrichissement de la liste des stopwords

In [ ]:
print(keywords)

## 2. Named Entity Recognition